# Lab Exercise 2

Exercise description in German.

## Kalman-Filter mit Messaussetzer

### Problemstellung

Für ein Fahrzeug soll die zurückgelegte Strecke abgeschätzt werden. Die Position des Fahrzeugs wird durch einen GPS-Sensor in äquidistant Zeitabständen erfasst. Ab und zu fährt das Fahrzeug durch in Tunnel, sodass die Position des Fahrzeugs zeitweise nicht erfassbar werden kann.

In [ ]:
%matplotlib widget
import pandas as pd

measurement_data = pd.read_csv(
    "data/Messaussetzer_data_in.dat",
    sep=" ",
)
measurement_data

In [ ]:
measurement_data.plot(
    y="position",
    kind="line",
    grid=True,
    xlabel="Zeit [s]",
    ylabel="Position [m]",
    title="Eingangsdaten",
)

### Aufgabenstellung

Entwerfen Sie ein geeignetes Kalman-Filter um zum einem die Position zu filtern und auf der anderen Seite, an den Stellen an denen das Fahrzeug im Tunnel ist, die Postion zu prädizieren.

Hinweis zur Bearbeitung: Untersuchen Sie verschiedene Modellansätze (Alpha-, Beta-, Gamma-Filter).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

y = measurement_data["position"].to_numpy()
u = np.zeros((y.size, 1))
t = measurement_data["time"].to_numpy()

Ts = t[1] - t[0]
non_nan = ~np.isnan(y)

#### Beta-Filter

$$\boldsymbol{x}(k + 1) = \begin{bmatrix}
1 & T_s \\
0 & 1 \\
\end{bmatrix} \cdot \begin{bmatrix}
s(k) \\
v(k)
\end{bmatrix} + \begin{bmatrix}
0 \\
0 \\
\end{bmatrix} \cdot \boldsymbol{u}(k) + \begin{bmatrix}
T_s \\
1 \\
\end{bmatrix} \cdot \boldsymbol{z}(k)$$

$$\boldsymbol{y}(k) = s(t) = \begin{bmatrix}
1 & 0 \\
\end{bmatrix} \cdot \begin{bmatrix}
s(t) \\
v(t) \\
\end{bmatrix} + \begin{bmatrix}
0 \\
\end{bmatrix} \cdot u(k)$$

In [ ]:
R = y[np.argwhere(non_nan == False)[-1][0] + 1 :].var()
Q = (5 / 200) ** 2
print(f"{R=:g}, {Q=:g}")

Ad = np.array([[1, Ts], [0, 1]])
Bd = np.array([[0], [0]])
C = np.array([[1, 0]])
D = 0
Gd = np.array([[Ts], [1]])

In [ ]:
x_hat = np.array([[y[0]], [0]])
P_hat = 30 * np.eye(2)

d_y = np.zeros_like(y)

s = np.zeros_like(y)
v = np.zeros_like(y)
K1 = np.zeros_like(y)
K2 = np.zeros_like(y)
P_hat1 = np.zeros_like(y)
P_hat2 = np.zeros_like(y)
P_hat3 = np.zeros_like(y)
P_hat4 = np.zeros_like(y)
for i, y_i in enumerate(y):
    if np.isnan(y_i):
        x_tilde = x_hat
        P_tilde = P_hat
    else:
        d_y[i] = y_i - (C @ x_hat + D * u[i])
        k = C @ P_hat @ C.T + R
        if k.ndim >= 2:
            K = P_hat @ C.T @ np.linalg.pinv(k)
        else:
            K = P_hat @ C.T / k
        x_tilde = x_hat + (K * d_y[i]).reshape(-1, 1)
        P_tilde = (np.eye(Bd.size) - K @ C) @ P_hat

    x_hat = Ad @ x_tilde + (Bd @ u[i]).reshape(-1, 1)
    P_hat = Ad @ P_tilde @ Ad.T + (Gd * Q) @ Gd.T

    s[i], v[i] = x_tilde
    K1[i], K2[i] = K
    P_hat1[i], P_hat2[i] = P_hat[0, :]
    P_hat3[i], P_hat4[i] = P_hat[1, :]
fig, (ax_s, ax_v) = plt.subplots(2, 1, figsize=(15, 7))

ax_s.plot(t, y, "k*")
ax_s.plot(t, s, "b--")
ax_s.plot(t, s + 3 * np.sqrt(P_hat1), "b-")
ax_s.plot(t, s - 3 * np.sqrt(P_hat1), "b-")
ax_s.grid(True)
ax_s.set_xlabel("Zeit [s]")
ax_s.set_ylabel("Position [m]")

ax_v.plot(t, v, "r-*")
ax_v.grid(True)
ax_v.set_xlabel("Zeit [s]")
ax_v.set_ylabel("Geschwindigkeit [m/s]")

fig.suptitle("Beta-Filter")
fig.tight_layout()

#### Alpha-Filter

$$x(k + 1) = \begin{bmatrix}
1 \\
\end{bmatrix} \cdot \begin{bmatrix}
s(k) \\
\end{bmatrix} + \begin{bmatrix}
0 \\
\end{bmatrix} \cdot u(k) + \begin{bmatrix}
1 \\
\end{bmatrix} \cdot z(k)$$
$$\boldsymbol{y}(k) = s(t) = \begin{bmatrix}
1 \\
\end{bmatrix} \cdot \begin{bmatrix}
s(t) \\
\end{bmatrix} + \begin{bmatrix}
0 \\
\end{bmatrix} \cdot u(k)$$

In [ ]:
R = y[np.argwhere(non_nan == False)[-1][0] + 1 :].var()
Q = np.diff(y[: np.argwhere(non_nan == False)[0][0]]).mean() ** 2
print(f"{R=:g}, {Q=:g}")

Ad = np.array([1])
Bd = np.array([0])
C = np.array([1])
D = np.array([0])
Gd = np.array([1])

In [ ]:
x_hat = np.array([y[0]])
P_hat = 30

d_y = np.zeros_like(y)

s = np.zeros_like(y)
K1 = np.zeros_like(y)
P_hat1 = np.zeros_like(y)
for i, y_i in enumerate(y):
    if np.isnan(y_i):
        x_tilde = x_hat
        P_tilde = P_hat
    else:
        d_y[i] = y_i - (C * x_hat + D * u[i])
        k = C * P_hat * C.T + R
        if k.ndim >= 2:
            K = P_hat * C.T * np.linalg.pinv(k)
        else:
            K = P_hat * C.T / k
        x_tilde = x_hat + (K * d_y[i]).reshape(-1, 1)
        P_tilde = (np.eye(Bd.size) - K * C) * P_hat

    x_hat = Ad * x_tilde + (Bd * u[i]).reshape(-1, 1)
    P_hat = Ad * P_tilde * Ad.T + (Gd * Q) * Gd.T

    s[i] = x_tilde
    K1[i] = K
    P_hat1[i] = P_hat
fig, ax_s = plt.subplots(1, 1, figsize=(15, 3.5))

ax_s.plot(t, y, "k*")
ax_s.plot(t, s, "b--")
ax_s.plot(t, s + 3 * np.sqrt(P_hat1), "b-")
ax_s.plot(t, s - 3 * np.sqrt(P_hat1), "b-")
ax_s.grid(True)
ax_s.set_xlabel("Zeit [s]")
ax_s.set_ylabel("Position [m]")

fig.suptitle("Alpha-Filter")
fig.tight_layout()

#### Gamma-Filter

$$\boldsymbol{x}(k + 1) = \begin{bmatrix}
1 & T_s & \frac{T_s^2}{2} \\
0 & 1 & T_s \\
0 & 0 & 1 \\
\end{bmatrix} + \begin{bmatrix}
0 \\
0 \\
0 \\
\end{bmatrix} \cdot \boldsymbol{u}(k) + \begin{bmatrix}
\frac{T_s^2}{2} \\
T_s \\
1 \\
\end{bmatrix} \cdot \boldsymbol{z}(k)$$

$$y(k) = \begin{bmatrix}
1 & 0 & 0 \\
\end{bmatrix} \cdot \begin{bmatrix}
s(k) \\
v(k) \\
a(k) \\
\end{bmatrix} + \begin{bmatrix}
0 \\
0 \\
0 \\
\end{bmatrix} \cdot \boldsymbol{u}(k)$$

In [ ]:
R = y[np.argwhere(non_nan == False)[-1][0] + 1 :].var()
Q = 0.0005
print(f"{R=:g}, {Q=:g}")

Ad = np.array([[1, Ts, 0.5 * Ts**2], [0, 1, Ts], [0, 0, 1]])
Bd = np.array([[0], [0], [0]])
C = np.array([[1, 0, 0]])
D = 0
Gd = np.array([[0.5 * Ts**2], [Ts], [1]])

In [ ]:
x_hat = np.array([[y[0]], [0], [0]])
P_hat = 30 * np.eye(3)

d_y = np.zeros_like(y)

s = np.zeros_like(y)
v = np.zeros_like(y)
a = np.zeros_like(y)
K1 = np.zeros_like(y)
K2 = np.zeros_like(y)
K3 = np.zeros_like(y)
P_hat1 = np.zeros_like(y)
P_hat2 = np.zeros_like(y)
P_hat3 = np.zeros_like(y)
P_hat4 = np.zeros_like(y)
P_hat5 = np.zeros_like(y)
P_hat6 = np.zeros_like(y)
P_hat7 = np.zeros_like(y)
P_hat8 = np.zeros_like(y)
P_hat9 = np.zeros_like(y)
for i, y_i in enumerate(y):
    if np.isnan(y_i):
        x_tilde = x_hat
        P_tilde = P_hat
    else:
        d_y[i] = y_i - (C @ x_hat + D * u[i])
        k = C @ P_hat @ C.T + R
        if k.ndim >= 2:
            K = P_hat @ C.T @ np.linalg.pinv(k)
        else:
            K = P_hat @ C.T / k
        x_tilde = x_hat + (K * d_y[i]).reshape(-1, 1)
        P_tilde = (np.eye(Bd.size) - K @ C) @ P_hat

    x_hat = Ad @ x_tilde + (Bd @ u[i]).reshape(-1, 1)
    P_hat = Ad @ P_tilde @ Ad.T + (Gd * Q) @ Gd.T

    s[i], v[i], a[i] = x_tilde
    K1[i], K2[i], K3[i] = K
    P_hat1[i], P_hat2[i], P_hat3[i] = P_hat[0, :]
    P_hat4[i], P_hat5[i], P_hat6[i] = P_hat[1, :]
    P_hat7[i], P_hat8[i], P_hat9[i] = P_hat[2, :]
fig, (ax_s, ax_v, ax_a) = plt.subplots(3, 1, figsize=(15, 7))

ax_s.plot(t, y, "k*")
ax_s.plot(t, s, "b--")
ax_s.plot(t, s + 3 * np.sqrt(P_hat1), "b-")
ax_s.plot(t, s - 3 * np.sqrt(P_hat1), "b-")
ax_s.grid(True)
ax_s.set_xlabel("Zeit [s]")
ax_s.set_ylabel("Position [m]")

ax_v.plot(t, v, "r-*")
ax_v.grid(True)
ax_v.set_xlabel("Zeit [s]")
ax_v.set_ylabel("Geschwindigkeit [m/s]")

ax_a.plot(t, a, "g-*")
ax_a.grid(True)
ax_a.set_xlabel("Zeit [s]")
ax_a.set_ylabel("Beschleunigung [m/s²]")

fig.suptitle("Gamma-Filter")
fig.tight_layout()